In [1]:
import numpy as np
import pandas as pd
import json
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
df_cc = pd.read_csv("data/creditcard.csv")
df_cc['normAmount'] = StandardScaler().fit_transform(df_cc['Amount'].values.reshape (-1,1))
df_cc = df_cc.drop(['Time', 'Amount'], axis = 1)
df_cc.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [3]:
def data_import(d, n, output, type, drop_amount, standardise):

    if (d != None) & (n != None):
        df_node = pd.read_csv(f"embeddings_{type}/vector_d{d}_n{n}.csv")

        lists = []

        for i in range(0, len(df_node)):
            string_list = df_node.embedding[i]
            lists.append(json.loads(string_list))

        nodes = []

        for i in range(0, len(lists[0])):
            nodes.append(f'N{i+1}')

        df_node[nodes] = pd.DataFrame(lists, index= df_node.index)

    if drop_amount == True:
        df_drop = df_cc.drop(['normAmount'], axis = 1)
    elif drop_amount == False:
        df_drop = df_cc

    if output == "NE":
        df = pd.concat([df_node.drop(['nodeId', 'embedding'], axis = 1), df_cc[['Class', 'normAmount']]], axis=1)
    elif output == "PCA":
        df = df_drop
    elif output == "ALL":
        df = pd.concat([df_node.drop(['nodeId', 'embedding'], axis = 1), df_drop], axis=1)

    if standardise == True:
        df.iloc[:, :-2] = StandardScaler().fit_transform(df.iloc[:, :-2])
    elif standardise == False:
        df = df

    return df

In [4]:
def data_preparation_classification(df):

    X = df.iloc[:, df.columns != 'Class']
    y = df.iloc[:, df.columns == 'Class']

    number_records_fraud = len(df[df.Class == 1])

    fraud_indices = np.array(df[df.Class==1].index)
    normal_indices = np.array(df[df.Class==0].index)

    random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False )
    under_sample_indices = np.concatenate ([fraud_indices, random_normal_indices])

    under_sample_df = df.iloc[under_sample_indices,:]
    X_undersample = under_sample_df.iloc [:, under_sample_df.columns != 'Class']
    y_undersample = under_sample_df.iloc [:, under_sample_df.columns == 'Class']

    X_train_under, X_test_under, y_train_under, y_test_under = train_test_split(X_undersample, y_undersample,
                                                                                test_size=0.3, random_state=None)

    LR = LogisticRegression(C=0.01, penalty='l2')
    LR.fit(X_train_under, y_train_under.values.ravel())
    y_pred = LR.predict(X_test_under)

    output_scores = [recall_score(y_test_under, y_pred), precision_score(y_test_under, y_pred), roc_auc_score(y_test_under, y_pred)]

    return output_scores

In [5]:
df_temporary = data_import(d=None, n=None, output="PCA", type="weighted", drop_amount=False, standardise=True)

In [6]:
#iteration
iteration = range(1, 51)
#outcome variables
recall = [None]
precision = [None]
AUC = [None]

lp1, lp2, lp3, lp4 = pd.core.reshape.util.cartesian_product([iteration, recall, precision, AUC])
bm = pd.DataFrame(dict(i=lp1, recall=lp2, precision=lp3, AUC=lp4))

In [7]:
for i in iteration:

    bm.loc[bm["i"] == i, ["recall", "precision", "AUC"]] = data_preparation_classification(df_temporary)

In [8]:
bm

,i,recall,precision,AUC
0,1,0.888889,0.992248,0.941155
1,2,0.839161,1.0,0.91958
2,3,0.869863,0.976923,0.924932
3,4,0.863014,0.984375,0.92484
4,5,0.861111,0.992,0.927266
5,6,0.93662,0.992537,0.965063
6,7,0.902778,0.970149,0.938231
7,8,0.81295,0.982609,0.900105
8,9,0.844156,0.977444,0.911515
9,10,0.892405,0.986014,0.938956


In [9]:
bm.to_csv(f"results/model/bm_LR_model.csv", index=False)